In [1]:
import sys
# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install --upgrade tfx[kfp]==0.30.0

     |████████████████████████████████| 2.4 MB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 15.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 25.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 31.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 43.3 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 4.2 MB/s  eta 0:00:01
     |████████████████████████████████| 17.7 MB 43.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 40.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 57.0 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 7.9 MB/s  eta 0:00:01
     |████████████████████████████████| 147 kB 40.4 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 103 kB 55.2 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 43 kB/s s eta 0:00:01
     |████████████████████████████████

In [3]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

env: GOOGLE_CLOUD_PROJECT=astute-pride-317802
GCP project ID:astute-pride-317802


In [4]:
# This refers to the KFP cluster endpoint
ENDPOINT='314cd0b9ca67984e-dot-asia-east1.pipelines.googleusercontent.com' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

In [5]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

In [9]:
PIPELINE_NAME="data_centric_pipeline"
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"data-centric",PIPELINE_NAME)

## Testing models

In [12]:
!{sys.executable} -m models.preprocessing_test
!{sys.executable} -m models.keras.model_test


2021-06-25 23:40:56.219453: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Running tests under Python 3.7.10: /opt/conda/bin/python
[ RUN      ] PreprocessingTest.testPreprocessingFn
INFO:tensorflow:time(__main__.PreprocessingTest.testPreprocessingFn): 0.0s
I0625 23:40:58.301542 140712543123264 test_util.py:2076] time(__main__.PreprocessingTest.testPreprocessingFn): 0.0s
[       OK ] PreprocessingTest.testPreprocessingFn
[ RUN      ] PreprocessingTest.test_session
[  SKIPPED ] PreprocessingTest.test_session
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK (skipped=1)
/opt/conda/bin/python: No module named models.keras.model_test


## Create first TFX pipeline
copy data to storage
execute after convert data into tfrecords (interactive notebook)

In [34]:
!gsutil -m cp -r data/train/ gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/data-centric/data/train/

Copying file://data/train/i.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/train/iv.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/train/x.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/train/vi.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/train/ix.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/train/v.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/train/iii.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/train/viii.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/train/ii.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/train/vii.tfrecords [Content-Type=application/octet-stream]...
/ [10/10 files][814.2 KiB/814.2 KiB] 100% Done                                  
Operation completed over 10 objects/814.2 KiB.                                   


In [32]:
!gsutil -m cp -r data/val/ gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/data-centric/data/val/

Copying file://data/val/x.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/val/vi.tfrecords [Content-Type=application/octet-stream]... 
Copying file://data/val/i.tfrecords [Content-Type=application/octet-stream]...  
Copying file://data/val/ix.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/val/iii.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/val/iv.tfrecords [Content-Type=application/octet-stream]... 
Copying file://data/val/ii.tfrecords [Content-Type=application/octet-stream]... 
Copying file://data/val/v.tfrecords [Content-Type=application/octet-stream]...  
Copying file://data/val/viii.tfrecords [Content-Type=application/octet-stream]...
Copying file://data/val/vii.tfrecords [Content-Type=application/octet-stream]...
/ [10/10 files][250.2 KiB/250.2 KiB] 100% Done                                  
Operation completed over 10 objects/250.2 KiB.                                   


## build docker image

In [51]:
!tfx pipeline create  --pipeline-path=kubeflow_runner.py --endpoint={ENDPOINT} \
--build-image

2021-06-26 03:50:05.301928: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 314cd0b9ca67984e-dot-asia-east1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
[Docker] Step 1/4 : FROM tensorflow/tfx:0.30.0[Docker] 
[Docker]  ---> 68adb9229d27
[Docker] Step 2/4 : WORKDIR /pipeline[Docker] 
[Docker]  ---> Using cache
[Docker]  ---> 62adad88abd0
[Docker] Step 3/4 : COPY ./ ./[Docker] 
[Docker]  ---> 5a2d0f74d12b
[Docker] Step 4/4 : ENV PYTHONPATH="/pipeline:${PY

## Create Pipeline
check kubeflow dashboard

In [49]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

2021-06-26 02:13:34.277943: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
CLI
Creating a run for pipeline: data_centric_pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 314cd0b9ca67984e-dot-asia-east1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
Run created for pipeline: data_centric_pipeline
+=======================+======================================+========+===========================+============================================================================================================================+
| pipeline_name         | run_id                               | status | created_at                | link                                                                                     

In [53]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

2021-06-26 04:01:58.266042: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 314cd0b9ca67984e-dot-asia-east1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generated pipeline:
 pipeline_info {
  id: "data_centric_pipeline"
}
nodes {
  pipeline_node {
    node_info {
      type {
        name: "tfx.components.example_gen.import_example_gen.component.ImportExampleGen"
      }
      id: "ImportExampleGen"
    }
    contexts {
      contexts {
      